In [43]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import FinanceDataReader as fdr

In [2]:
BASE_URL = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok='
req = requests.get(BASE_URL)
req

<Response [200]>

In [44]:
BASE_URL = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok='

KOSPI_CODE = 0
KOSDAK_CODE = 1
START_PAGE = 1
field = []

def main(code):
    res = requests.get(BASE_URL + str(code) + '&page=' + str(START_PAGE))
    page_soup = BeautifulSoup(res.text, 'html')
    
    total_page_num = page_soup.select_one('td.pgRR > a')
    total_page_num = int(total_page_num.get('href').split('=')[-1])
    
    ipt_html = page_soup.select_one('div.subcnt_sise_item_top')
    global fields
    fields = [item.get('value') for item in ipt_html.select('input')]
    print('field 첫번째 값들', fields)
    
    result = [crawl(code,str(page)) for page in range(1, total_page_num + 1)]
    print('result 값들',result)
    
    df = pd.concat(result, axis = 0, ignore_index = True)
    return df
    
def crawl(code, page):
    global fields
    data = {'menu': 'market_sum',
            'fieldIds': fields,
            'returnUrl': BASE_URL + str(code) + "&page=" + str(page)}
    res = requests.post('https://finance.naver.com/sise/field_submit.nhn', data = data)
    page_soup = BeautifulSoup(res.text, 'lxml')
    table_html = page_soup.select_one('div.box_type_l')
    
    header_data = [item.get_text().strip() for item in table_html.select('thead th')][1:-1]
    
    inner_data = [item.get_text().strip() for item in table_html.find_all(lambda x:
                                                                            (x.name == 'a' and
                                                                             'tltle' in x.get('class', [])) or
                                                                            (x.name == 'td' and
                                                                             'number' in x.get('class', []))
                                                                            )]
    
    no_data = [item.get_text().strip() for item in table_html.select('td.no')]
    
    number_data = np.array(inner_data)
    #print(f'number_data 값 {number_data}')
    
    number_data.resize(len(no_data), len(header_data))
    
    df = pd.DataFrame(data = number_data, columns = header_data)
    return df
main(KOSPI_CODE)


field 첫번째 값들 ['quant', 'ask_buy', 'amount', 'market_sum', 'operating_profit', 'per', 'open_val', 'ask_sell', 'prev_quant', 'property_total', 'operating_profit_increasing_rate', 'roe', 'high_val', 'buy_total', 'frgn_rate', 'debt_total', 'net_income', 'roa', 'low_val', 'sell_total', 'listed_stock_cnt', 'sales', 'eps', 'pbr', 'sales_increasing_rate', 'dividend', 'reserve_ratio']


KeyboardInterrupt: 

###현재 네이버에 기재된 매출액, 총자산은 마지막 재무제표 결산일을 기준으로 한다 

In [10]:
KOSPI = main(KOSPI_CODE)
KOSDAK = main(KOSDAK_CODE)

In [11]:
KOSPI = KOSPI.replace('N/A',np.nan).dropna(axis = 0)
KOSPI['PER'] = KOSPI['PER'].str.replace(',','').astype('float64')
KOSPI['ROE'] = KOSPI['ROE'].str.replace(',','').astype('float64')
KOSPI['ROA'] = KOSPI['ROA'].str.replace(',','').astype('float64')
KOSPI['시가총액'] = KOSPI['시가총액'].str.replace(',','').astype('int64')
KOSPI['매출액'] = KOSPI['매출액'].str.replace(',','').astype('int64')
KOSPI['영업이익'] = KOSPI['영업이익'].str.replace(',','').astype('int64')
KOSPI['PSR'] = KOSPI['시가총액'] / KOSPI['매출액']
KOSPI['POR'] = KOSPI['시가총액'] / KOSPI['영업이익']
KOSPI = KOSPI[['종목명','PER','ROE','ROA','PBR','PSR','POR']]
c = ['PER','POR']
KOSPI[c] = KOSPI[c].mask(KOSPI[c] < 0)
KOSPI = KOSPI.dropna(axis = 0)

In [12]:
KOSPI['PBR + PSR'] = KOSPI[['PBR','PSR']].rank(method = 'min').sum(axis = 1)
KOSPI['PBR + PSR + POR'] = KOSPI[['PBR','PSR','POR']].rank(method = 'min').sum(axis = 1)
KOSPI['total_rank'] = (KOSPI[['PER','PBR','PSR']].rank(method = 'min').sum(axis = 1) + KOSPI[['ROE','ROA']].rank(method = 'min',ascending = False).sum(axis = 1))

In [14]:
PBR_PSR = KOSPI[['종목명','PBR','PSR','PBR + PSR']].sort_values(by = 'PBR + PSR')
PBR_PSR = PBR_PSR.drop(['PBR + PSR'],axis = 1)
PBR_PSR.index = list(range(1,len(PBR_PSR)+1))
PBR_PSR = round(PBR_PSR,2)
PBR_PSR.to_excel('PBR+PSR.xlsx')

In [15]:
PBR_PSR_POR = KOSPI[['종목명','PBR','PSR','POR','PBR + PSR + POR']].sort_values(by = 'PBR + PSR + POR')
PBR_PSR_POR = PBR_PSR_POR.drop(['PBR + PSR + POR'],axis = 1)
PBR_PSR_POR.index = list(range(1,len(PBR_PSR_POR)+1))
PBR_PSR_POR = round(PBR_PSR_POR,2)
PBR_PSR_POR.to_excel('PBR+PSR+POR.xlsx')

In [16]:
TOTAL = KOSPI.sort_values(by = 'total_rank')
TOTAL = TOTAL.drop(['PBR + PSR','PBR + PSR + POR','total_rank'], axis = 1)
TOTAL.index = list(range(1,len(TOTAL)+1))
TOTAL = round(TOTAL,2)
TOTAL.to_excel('TOTAL.xlsx')

In [17]:
TOTAL

,종목명,PER,ROE,ROA,PBR,PSR,POR
1,사조대림,8.04,30.57,9.72,0.33,0.10,2.67
2,한신공영,2.23,15.21,4.20,0.31,0.10,1.33
3,디티알오토모티브,4.02,11.50,7.66,0.32,0.24,2.79
4,KG케미칼,2.21,32.69,7.75,0.75,0.14,2.43
5,계룡건설,2.84,16.94,4.00,0.43,0.09,1.56
...,...,...,...,...,...,...,...
439,삼일제약,223.21,1.33,0.47,2.41,1.35,34.02
440,보락,251.88,1.48,1.19,2.76,3.34,201.17
441,녹십자,502.94,-0.35,-0.61,2.82,2.19,74.53
442,신풍제약,4272.73,0.80,0.46,33.11,39.38,3735.45


In [11]:
KOSPI = fdr.StockListing('KOSPI')
KOSDAQ = fdr.StockListing('KOSDAQ')
Symbol = KOSDAQ['Symbol']
Name = KOSDAQ['Name']
target = [ i for i in zip(Symbol,Name)]
target = pd.DataFrame(target)
target.columns = ['Symbol','Name']

### 코스닥 상위 종목

In [12]:
ALL = pd.read_excel('all case_v2.xlsx')
c = ['매출액','PER(배)','PBR(배)']
ALL[c] = ALL[c].mask(ALL[c] <= 0)
ALL = ALL.dropna(axis = 0)
ALL.index = list(range(1,len(ALL)+1))
ALL = ALL.drop(['당기순이익','영업이익','EPS(원)','발행주식수(보통주)','total_rank'],axis = 1)
ALL

,종목명,PER(배),PBR(배),ROE(%),ROA(%),매출액,시가총액,PSR,POR
1,이글루시큐리티,8.02,1.24,17.61,11.71,771.0,503.622250,0.653207,3.965530
2,우리넷,5.05,0.62,12.85,9.43,496.0,400.403159,0.807264,6.458115
3,삼영엠텍,8.02,1.24,17.61,11.71,771.0,514.800000,0.667704,4.053543
4,코위버,5.16,0.55,12.01,9.93,600.0,798.439200,1.330732,12.878052
5,시큐브,5.05,0.62,12.85,9.43,496.0,591.600000,1.192742,9.541935
...,...,...,...,...,...,...,...,...,...
201,진성티이씨,236.12,3.70,1.66,0.59,439.0,2371.879274,5.402914,50.465516
202,안랩,257.11,1.16,0.46,0.39,627.0,5377.440135,8.576460,215.097605
203,태광,236.12,3.70,1.66,0.59,439.0,3193.250000,7.273918,67.941489
204,한글과컴퓨터,155.11,10.00,6.66,4.45,124.0,4143.256874,33.413362,517.907109


### 테마별 수익률 (당일 계속 크롤링)

In [13]:
url = 'https://finance.naver.com/sise/sise_group.nhn?type=upjong'
req = requests.get(url)
session = requests.session()
res = session.get(url)

html = req.text
soup = BeautifulSoup(html, 'html.parser')
a = soup.select('div > #contentarea > #contentarea_left > table > tr > td > a')

name = []
for i in range(0,79):
    title = a[i].get_text().strip()
    name.append(title)

b = soup.select('div > #contentarea > #contentarea_left > table > tr > td > span')

value = []
for i in range(0,79):
    title = b[i].get_text().strip()
    value.append(title)
target1 = [ i for i in zip(name,value)]
target1 = pd.DataFrame(target1)
target1.columns = ['테마명','등락률']

target1.to_excel('실시간 테마별 수익.xlsx')

In [14]:
target1.head(10)

,테마명,등락률
0,독립전력생산및에너지거래,+29.43%
1,소프트웨어,+9.95%
2,석유와가스,+4.26%
3,해운사,+4.12%
4,출판,+3.42%
5,전기장비,+3.24%
6,전자제품,+3.03%
7,에너지장비및서비스,+2.56%
8,식품,+2.31%
9,건강관리기술,+2.27%


### 실시간 뉴스 타이틀 크롤링

In [ ]:
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup

plusUrl = urllib.parse.quote_plus(input('검색어를 입력하세요:'))

pageNum = 1
count = 1

i = input('몇페이지 크롤링 할까요? : ')

lastPage = int(i) * 10 - 9
while pageNum < lastPage + 1:
    url = f'https://search.naver.com/search.naver?date_from=&date_option=0&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={plusUrl}&sm=tab_pge&srchby=all&st=sim&where=post&start={pageNum}'

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find_all(class_='sh_blog_title')

    print(f'-----{count}페이지 결과입니다.-----')
    for i in title:
        print(i.attrs['title'])
        print(i.attrs['href'])
    print()
        
    pageNum += 10
    count += 1

In [45]:
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup

plusUrl = urllib.parse.quote_plus(input('검색어를 입력하세요:'))

pageNum = 1
count = 1

i = 1

lastPage = int(i) * 10 - 9
while pageNum < lastPage + 1:
    url = f'https://search.naver.com/search.naver?date_from=&date_option=0&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=&query={plusUrl}&sm=tab_pge&srchby=all&st=sim&where=post&start={pageNum}'

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find_all(class_='sh_blog_title')

    print(f'-----{count}페이지 결과입니다.-----')
    for i in title:
        print(i.attrs['title'])
        print(i.attrs['href'])
    print()
        
    pageNum += 10
    count += 1

검색어를 입력하세요: 삼성


-----1페이지 결과입니다.-----
갤럭시는 삼성 UV 살균기로 소독해야 제맛
https://blog.naver.com/tech-plus?Redirect=Log&logNo=222031519131
삼성생명 여성시대보험 좋으나 추가필요해요!
https://blog.naver.com/mnbvcxzlkjgf?Redirect=Log&logNo=222083012706
신한 삼성 우리 국민 화물복지카드 사용시 알아야할점
https://blog.naver.com/titimu?Redirect=Log&logNo=222050316920
럭셔리 호캉스 끝판왕 신라스테이삼성
https://blog.naver.com/i7866?Redirect=Log&logNo=222078461363
삼성공채 GSAT인강 이용해 채용 준비하기!
https://blog.naver.com/e_yura?Redirect=Log&logNo=222085550848
9월 7일의 기업분석 Letter - 삼성전자(005930)
https://blog.naver.com/ionia17?Redirect=Log&logNo=222082684896
삼성동 아파트 브르넨삼성 신규분양문의
http://codeone.co.kr/222013265268
2010년에 삼성전자에 투자했었더라면.
https://blog.naver.com/how2invest?Redirect=Log&logNo=222077219143
논현동빌라경매 강남구 논현동79-5 논현삼성빌라트
https://blog.naver.com/k7k8h1009?Redirect=Log&logNo=222078165598
이제 다시 서울로, 이제 다시 삼성전자로
http://ohboxa.com/222014771651

